In [56]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models, losses, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [57]:
def make_model(
                filters = 32,
                dense_inner_size = 64,
                lr = 0.002,
                momentum = 0.8,
                loss = tf.keras.losses.BinaryCrossentropy()
                ):
    inputs = keras.Input(shape=(200, 200, 3))
    
    # Convolutional layers
    cvl_layers = layers.Conv2D(filters=filters, kernel_size=(3, 3), activation='relu')(inputs)
    feature_map = layers.MaxPooling2D(pool_size=(2, 2))(cvl_layers)
    
    # Flatten layer
    vectors = layers.Flatten()(feature_map)
    
    # Dense layers
    dense_layers = layers.Dense(dense_inner_size, activation='relu')(vectors)
    
    # Output layer for binary classification
    outputs = layers.Dense(1, activation='sigmoid')(dense_layers)
    
    # Model construction
    model = keras.Model(inputs, outputs)
    
    # Optimizer and compiling the model
    optimizer = optimizers.SGD(learning_rate=lr, momentum=momentum)
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [58]:
model = make_model()

In [59]:
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 conv2d_13 (Conv2D)          (None, 198, 198, 32)      896       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 99, 99, 32)       0         
 g2D)                                                            
                                                                 
 flatten_13 (Flatten)        (None, 313632)            0         
                                                                 
 dense_24 (Dense)            (None, 64)                20072512  
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                          

In [60]:
train_gen = ImageDataGenerator(rescale=1./255)


train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(200, 200), # Size of the image
    class_mode = 'binary',
    shuffle=True,
    batch_size=20  # How many image at once we need
)
test_gen = ImageDataGenerator(rescale=1./255)
test_ds = test_gen.flow_from_directory(
    './data/test',
    target_size=(200, 200), # Size of the image
    class_mode = 'binary',
    shuffle=True,
    batch_size=20  # How many image at once we need
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [61]:
model = make_model()
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10
40/40 [==============================] - 9s 212ms/step - loss: 0.7177 - accuracy: 0.5375 - val_loss: 0.6527 - val_accuracy: 0.5871
Epoch 2/10
40/40 [==============================] - 8s 201ms/step - loss: 0.6184 - accuracy: 0.6475 - val_loss: 0.6283 - val_accuracy: 0.6318
Epoch 3/10
40/40 [==============================] - 8s 210ms/step - loss: 0.5833 - accuracy: 0.6888 - val_loss: 0.6189 - val_accuracy: 0.6617
Epoch 4/10
40/40 [==============================] - 8s 211ms/step - loss: 0.5831 - accuracy: 0.6888 - val_loss: 0.6168 - val_accuracy: 0.6368
Epoch 5/10
40/40 [==============================] - 8s 208ms/step - loss: 0.5478 - accuracy: 0.7175 - val_loss: 0.6168 - val_accuracy: 0.6418
Epoch 6/10
40/40 [==============================] - 8s 210ms/step - loss: 0.5242 - accuracy: 0.7412 - val_loss: 0.6650 - val_accuracy: 0.6368
Epoch 7/10
40/40 [==============================] - 8s 207ms/step - loss: 0.5125 - accuracy: 0.7412 - val_loss: 0.6056 - val_accuracy: 0.6418
Epoch 

In [64]:
np.median(history.history['accuracy'])

0.7293749749660492

In [63]:
np.std(history.history['loss'])


0.07327200425326183